![spotify_logo](images/spotify_logo2.1.png)

# Spotify API Scrape

## Project Goal
1. Pull most recently played tracks
2. Pull top listen to tracks

## Helpful Links:
- [Spotify Web API - Authorization Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)
- [Spotify API References](https://developer.spotify.com/documentation/web-api/reference/)

In [ ]:
import config
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode

import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import xlsxwriter
from tzlocal import get_localzone

In [ ]:
# Create the file name we will be exporting later
today = datetime.today().strftime('%Y%m%d')
file_name = f"Spotify_Export_{today}.xlsx"
file_name

## Step 1: Get Access Token

In [ ]:
client_id = config.client_id
client_secret = config.client_secret

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

## Step 2: Pull Recently Played

In [ ]:
base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_data.append(json.loads(r.text))

In [ ]:
track_ids = []
album_names = []
artist_names = []
track_names = []
played_ats = []

for i in range(len(personal_data[0]['items'])):
    track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
    album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
    artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
    track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name
    played_ats.append(personal_data[0]['items'][i]['played_at'])

In [ ]:
list_dic={'track_id':track_ids,
          'track_name':track_names,
          'artist_name':artist_names,
          'album_name':album_names,
          'time_palyed': played_ats
    }

In [ ]:
df1=pd.DataFrame(list_dic)
df1

In [ ]:
# Convert UTC to local time zone
utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[0]).astimezone(get_localzone())
# Format date/time
local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
local_time

In [ ]:
# Create empty column to append data to
df1['local_time'] = ''

for i in range(len(personal_data[0]['items'])):
    utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[i]).astimezone(get_localzone())
    # Format date/time
    local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
    local_time
    
    df1['local_time'].iloc[i] = local_time

In [ ]:
df1.head()

In [ ]:
writer = pd.ExcelWriter(file_name)
df1.to_excel(writer,'recently_played')

## Step 3: Pull User's Top Artists & Tracks

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-top-read'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [ ]:
base_url = 'https://api.spotify.com/v1/me/top/tracks?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
top_track_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"time_range=medium_term"+"&&limit=50", headers=headers)
top_track_data.append(json.loads(r.text))

In [ ]:
track_idss = []
album_namess = []
album_relase_datess = []
artist_namess = []
track_namess = []

for i in range(len(top_track_data[0]['items'])):
    track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
    album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
    album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
    artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
    track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name

In [ ]:
list_dic2={'track_id':track_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'album_relase_date':album_relase_datess,
    }

In [ ]:
df2=pd.DataFrame(list_dic2)
df2

## Step 4: Pull Track Info
- API Doc: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

In [ ]:
track_id_ls = df2['track_id'].tolist()

In [ ]:
track_data = []
for id in tqdm(track_id_ls):
    base_url = f'https://api.spotify.com/v1/audio-features/{id}?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    r = requests.get(base_url, headers=headers)
    track_data.append(json.loads(r.text))

In [ ]:
track_df = pd.json_normalize(track_data)
track_df.head()

## Clean Up Track Data

## Update 'mode' to tell if track is major or minor

In [ ]:
mod_dict = {0 : 'Minor',
            1: 'Major'}

In [ ]:
track_df['mode'].replace(mod_dict, inplace=True)

## Update "key" to tell the actually key

In [ ]:
music_dic = {
    0: 'C',
    1: 'C#/Db',
    2: 'D',
    3: 'D#/Eb',
    4: 'E',
    5: 'F',
    6: 'F#/Gb',
    7: 'G',
    8: 'G#/Ab',
    9: 'A',
    10: 'A#/Bb',
    11: 'B' 
}

In [ ]:
track_df['key'].replace(music_dic, inplace=True)

## Add column duration in minutes/seconds

In [ ]:
track_df['track_duration'] = ''

for i in range(len(track_df['id'])):
    millis=track_df['duration_ms'].iloc[i]
    track_df['track_duration'].iloc[i] = pd.to_datetime(millis, unit='ms').strftime('%H:%M:%S')

## Rename track id column

In [ ]:
track_df=track_df.rename(columns = {'id':'track_id'})
track_df.head()                     

# Create Master Dataframe

In [ ]:
master_df = pd.merge(df2, track_df,
                       how='left', on=['track_id'])

In [ ]:
master_df.head(10)

In [ ]:
master_df.to_excel(writer,'top_tracks')

# Create Excel File

In [ ]:
current_directory = os.path.abspath(os.getcwd())

In [ ]:
export_file_path = os.path.join(current_directory, 'spotify_export_files', file_name)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(export_file_path, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer,'recently_played', index=False)
master_df.to_excel(writer,'top_tracks',index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet1 = writer.sheets['recently_played']
worksheet2 = writer.sheets['top_tracks']

for i, col in enumerate(master_df.columns):

    # find length of column i
    column_len = master_df[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet1.set_column(i, i, column_len)
    worksheet2.set_column(i, i, column_len)
writer.save()